# Capacity and quantum geometry of parametrized quantum circuits


"Capacity and quantum geometry of parametrized quantum circuits" by T. Haug, K. Bharti, M.S. Kim
arXiv:2102.01659, PRX Quantum 2, 040309 (2021)

Example code to calculate quantum Fisher information matrix,effective quantum dimension and quantum natural gradient.

Calculates properties of a parametrized quantum circuit 
U(\theta)=W_L R_L(\theta) W_{L-1} R_{L-1}(\theta) ... W_1 R_1(\theta) \sqrt{H}\ket{0}

W_l is entangling layer of two-qubit entangling operations, R_l are single-qubit rotations

Prunes redundant parameters from circuit by using zero eigenvalues of quantum fisher information metric. 

Based on qutip

@author: Tobias Haug, github txhaug
Imperial College London
Contact at tobiasxhaug@gmail.com


In [1]:
import qutip as qt

from functools import partial

import operator
from functools import reduce
import numpy as np

import scipy

In [2]:
def prod(factors):
    return reduce(operator.mul, factors, 1)


def flatten(l):
    return [item for sublist in l for item in sublist]

#tensors operators together 
def genFockOp(op,position,size,levels=2,opdim=0):
    opList=[qt.qeye(levels) for x in range(size-opdim)]
    opList[position]=op
    return qt.tensor(opList)


##convert list of parameters into a 2D array
##removes unused parameters where ini_pauli=0
def construct_2d_parameters(angles,ini_pauli):
    depth,n_qubits=np.shape(ini_pauli)
    angles2d=np.zeros([depth,n_qubits])
    counter=0
    
    for i in range(depth):
        for j in range(n_qubits):
            if(ini_pauli[i,j]==0):
                pass
            elif(paulis2d[i,j]>0):
                ini_pauli[i,j]=angles[counter]
                counter+=1

    return angles2d

##convert 2D array of parameters into 1D list
def construct_1d_parameters(angles2d,paulis2d):
    depth,n_qubits=np.shape(paulis2d)
    angles1d=[]
    for i in range(depth):
        for j in range(n_qubits):
            if(paulis2d[i,j]>0):
                angles1d.append(angles2d[i,j])
    
    return np.array(angles1d)

In [3]:
#calculate state and gradients
def get_states_gradients(ini_pauli,ini_angles,n_parameters):
    #list of values of gradient
    gradient_list=np.zeros(n_parameters)

    #save here quantum state of gradient
    grad_state_list=[]


    #p goes from -1 to n_parameters-1. -1 is to calculate quantum state, 0 to n_parameters rest for gradient
    #calculates the quantum state U\ket{0}, as well as \partial_i U \ket{0}
    #those can be used to calcute energy, gradients and the Quantum Fisher metric 
    for p in range(-1,n_parameters):
        counter=0 #counter to keep track of which parameter is calculated
        initial_state=qt.tensor([qt.basis(levels,0) for i in range(n_qubits)])
        #initial layer of fixed \sqrt{H} rotations
        initial_state=qt.tensor([qt.qip.operations.ry(np.pi/4) for i in range(n_qubits)])*initial_state

        #go through depth layers
        for j in range(depth):
            rot_op=[]
            #define parametrized single-qubit rotations at layer j
            for k in range(n_qubits):
                if(ini_pauli[j][k]!=0):
                    angle=ini_angles[j][k]
                    if(ini_pauli[j][k]==1):
                        rot_op.append(qt.qip.operations.rx(angle))
                    elif(ini_pauli[j][k]==2):
                        rot_op.append(qt.qip.operations.ry(angle))
                    elif(ini_pauli[j][k]==3):
                        rot_op.append(qt.qip.operations.rz(angle))


                    #multiply in derivative of parametrized single-qubit rotation gate at layer j for parameter of circuit p
                    #this is the exact derivative
                    if(counter==p):
                        if(ini_pauli[j][k]==1):
                            initial_state=(-1j*opX[k]/2)*initial_state
                        elif(ini_pauli[j][k]==2):
                            initial_state=(-1j*opY[k]/2)*initial_state
                        elif(ini_pauli[j][k]==3):
                            initial_state=(-1j*opZ[k]/2)*initial_state



                    counter+=1
                else:
                    ##if not used, just add identity
                    rot_op.append(qt.qeye(2))

            #multiply in single-qbuit rotations 
            initial_state=qt.tensor(rot_op)*initial_state

            #add entangling layer
            initial_state=entangling_layer*initial_state

        if(p==-1):#get quantum state for cost function
            #cost function given by <\psi|H|\psi>
            circuit_state=qt.Qobj(initial_state)#state generated by circuit
            energy=qt.expect(H,circuit_state)


        else:#get quantum state needed for gradient
            grad_state_list.append(qt.Qobj(initial_state))#state with gradient applied for p-th parameter

            #gradient of circuit is given by 2*real(<\psi|H|\partial_p\psi>)
            gradient_list[p]=2*np.real(circuit_state.overlap(H*initial_state))
            
    return circuit_state,energy,grad_state_list,gradient_list

def calc_qfi(circuit_state,grad_state_list):
    #quantum fisher information metric
    #calculated as \text{Re}(\braket{\partial_i \psi}{\partial_j \psi}-\braket{\partial_i \psi}{\psi}\braket{\psi}{\partial_j \psi})


    #first, calculate elements \braket{\psi}{\partial_j \psi})
    single_qfi_elements=np.zeros(n_parameters,dtype=np.complex128)
    for p in range(n_parameters):
        #print(circuit_state.overlap(grad_state_list[p]))
        single_qfi_elements[p]=circuit_state.overlap(grad_state_list[p])


    #calculcate the qfi matrix
    qfi_matrix=np.zeros([n_parameters,n_parameters])
    for p in range(n_parameters):
        for q in range(p,n_parameters):
            qfi_matrix[p,q]=np.real(grad_state_list[p].overlap(grad_state_list[q])-np.conjugate(single_qfi_elements[p])*single_qfi_elements[q])


    #use fact that qfi matrix is real and hermitian
    for p in range(n_parameters):
        for q in range(p+1,n_parameters):  
            qfi_matrix[q,p]=qfi_matrix[p,q]
    return qfi_matrix

Set parameters here for circuit

In [4]:

n_qubits=4 #number qubits
depth=16 #number of layers

#type of entangling gate used
type_entanglers=1 #0: CNOT, 1:CPHASE, 2: \sqrt{iSWAP}

#how to arrange the entangling layer
entangling_arrangement=0 #0: one-dimensional nearest-neighbor CHAIN, 1: ALl-to-ALL connections


In [5]:

#random generator used
rng = np.random.default_rng(1)


#define angles for circuit
ini_angles=rng.random([depth,n_qubits])*2*np.pi

#define rotations for circuit in each layer, 1: X, 2:Y 3:Z
ini_pauli=rng.integers(1,4,[depth,n_qubits])


##test case for n_qubits=4, depth=3, type_entanglers=0,entangling_arrangement=0
#ini_angles=np.reshape([1.48304, 2.17723, 1.9648, 0.0496955, 3.07004, 1.32555, 5.98107, 6.28259, 1.58124, 6.19941, 3.49189, 2.74643],[depth,n_qubits])
#ini_pauli=np.reshape([3, 2, 2, 2, 3, 2, 3, 1, 3, 2, 2, 1],[depth,n_qubits])
##should give energy=-0.290501, gradient_mean= 0.112244, qng_mean=0.3352


n_parameters=len(construct_1d_parameters(ini_angles,ini_pauli)) #number of parameters of circuit


cutoff_eigvals=10**-12 #define all eigenvalues of quantum fisher information metric as 0

#operators for circuit
levels=2#
opZ=[genFockOp(qt.sigmaz(),i,n_qubits,levels) for i in range(n_qubits)]
opX=[genFockOp(qt.sigmax(),i,n_qubits,levels) for i in range(n_qubits)]
opY=[genFockOp(qt.sigmay(),i,n_qubits,levels) for i in range(n_qubits)]
opId=genFockOp(qt.qeye(levels),0,n_qubits)
    

H=opZ[0]*opZ[1] #local Hamiltonian to calculate energy and gradient from
    

#define entangling gate arrangement, 
if(entangling_arrangement==0):    #here is for chain topology
    entangling_gate_index=[[2*j,2*j+1] for j in range(n_qubits//2)]+[[2*j+1,2*j+2] for j in range((n_qubits-1)//2)]
elif(entangling_arrangement==1):##all-to-all
    #randomize control and target for more entangling power for CNOT
    entangling_gate_index=flatten([[rng.permutation([i,j]) for j in range(i+1,n_qubits)] for i in range(n_qubits-1)])


#type of entangliers used
if(type_entanglers==0):#CNOT
    entangling_layer=prod([qt.qip.operations.cnot(n_qubits,j,k) for j,k in entangling_gate_index][::-1])#need [::-1] to invert order so that unitaries are multiplied in correct order
elif(type_entanglers==1):#CPHASE
    entangling_layer=prod([qt.qip.operations.csign(n_qubits,j,k) for j,k in entangling_gate_index][::-1])
elif(type_entanglers==2):#\sqrt{iSWAP}
    entangling_layer=prod([qt.qip.operations.sqrtiswap(n_qubits,[j,k]) for j,k in entangling_gate_index][::-1])



In [6]:
circuit_state,energy,grad_state_list,gradient_list=get_states_gradients(ini_pauli,ini_angles,n_parameters)
print("Energy of state",energy)

Energy of state -0.016312378471624672


In [7]:
qfi_matrix=calc_qfi(circuit_state,grad_state_list)

Get eigenvalues and eigenvectors of quantum fisher information metric. Gives information about the parameter space of the circuit. Zero eigenvalues indicate redundant parameters, i.e. which do not contribute The larger the eigenvalue, the more the quantum state changes when changing the parameters in the direction of the eigenvector-
For random initial parameters, the effective quantum dimension is equivalent to the parameter dimension, which is the total number of independent parameters that the quantum state generated by the quantum circuit can represent.

In [8]:
#get eigenvalues and eigenvectors of QFI
eigvals,eigvecs=scipy.linalg.eigh(qfi_matrix)

In [9]:
#non-zero eigenvalues of qfi
nonzero_eigvals=eigvals[eigvals>cutoff_eigvals]

#effective quantum dimension, i.e. number of independent directions when perturbing system
#is equivalent to parameter dimension (number of independent parameters of quantum state that can be represented by circuit) for random initial values of circuit
eff_quant_dim=len(nonzero_eigvals)

n_zero_eigval=n_parameters-eff_quant_dim

print("Hilbert space", 2**n_qubits)
print("Number parameters of circuit",n_parameters)
print("Effective quantum dimension G_C",eff_quant_dim)
print("Number of zero eigenvalues of QFI",n_zero_eigval)

Hilbert space 16
Number parameters of circuit 64
Effective quantum dimension G_C 30
Number of zero eigenvalues of QFI 34


Number of parameters of circuit that are redundant

In [10]:
#fraction of zero eigenvalues
redundancy=n_zero_eigval/n_parameters
print("redundancy",redundancy)

redundancy 0.53125


In [11]:
##get sum of amplitudes of eigenvectors with zero eigenvalue
##tells us how redundant a parameter is
weights_zeros=np.zeros(n_parameters)
for i in range(n_parameters):
    #only get weights for zero eigvals
    if(eigvals[i]<cutoff_eigvals):
        weights_zeros[:]+=np.abs(eigvecs[:,i])**2

The quantum Fisher information metric (QFI) is a matrix telling us how a change in parameter of the quantum circuit affects the fidelity of the quantum state. 
The eigenvectors of the QFI tell us the directions in parameter space that lead to a change in fidelity given by the corresponding eigenvalues. If the eigenvalue is zero, the corresponding eigenvector is a direction in parameter space that does not change the quantum state. 
The algorithm calculates the eigenvectors with eigenvalue zero, and iteratively removes the parameters that have no effect on the quantum state. This is done by checking which parameters of the eigenvector have non-zero amplitude, and removing them from the circuit.

In [18]:
##find which parameters can be pruned
reduced_parameters=n_parameters
reduced_eigvecs=np.array(eigvecs)
reduced_eigvals=np.array(eigvals)
reduced_weights_zeros=np.array(weights_zeros)
reduced_qfi_matrix=np.array(qfi_matrix)
removed_index=[]
epsilon=10**-14
reduced_n_zero_eigval=n_parameters-eff_quant_dim
while(reduced_n_zero_eigval>0):

    reduced_zero_eigvals_index=np.arange(reduced_parameters)[reduced_weights_zeros>epsilon]
    delete_index=reduced_zero_eigvals_index[-1]

    removed_index.append(delete_index)#always delete last index, so counting is fine
    reduced_qfi_matrix=np.delete(reduced_qfi_matrix,delete_index,axis=0)
    reduced_qfi_matrix=np.delete(reduced_qfi_matrix,delete_index,axis=1)
    reduced_parameters-=1
    reduced_eigvals,reduced_eigvecs=scipy.linalg.eigh(reduced_qfi_matrix)

    reduced_weights_zeros=np.zeros(reduced_parameters)
    for i in range(reduced_parameters):
        #only get weights for zero eigvals
        if(reduced_eigvals[i]<cutoff_eigvals):
            reduced_weights_zeros[:]+=np.abs(reduced_eigvecs[:,i])**2

    reduced_n_zero_eigval=0
    for i in range(reduced_parameters):
        if(reduced_eigvals[i]<cutoff_eigvals):
            reduced_n_zero_eigval+=1

    #reconstruct weight plot with deleted indices
    reduced_weights_zeros_plot=np.array(reduced_weights_zeros)
    for i in range(len(removed_index)): 
        reduced_weights_zeros_plot=np.insert(reduced_weights_zeros_plot,removed_index[len(removed_index)-1-i],-1)
    reduced_weights_zeros_plot=np.reshape(reduced_weights_zeros_plot,[depth,n_qubits])
    #plot2D(reduced_weights_zeros_plot,qubit_range,depth_range,dataset+"weightszerosCut",saveto,"qubits","depth")


In [13]:
##remove removed parameters from circuit by setting corresponding pruned_ini_pauli=0
counter=0
pruned_ini_pauli=np.array(ini_pauli)
for i in range(depth):
    for j in range(n_qubits):
        if(ini_pauli[i,j]!=0):
            if(counter in removed_index): #if has been removed by pruning, set pauli rotation to zero
                pruned_ini_pauli[i,j]=0
            counter+=1
            


In [14]:
##pruned pauli rotations of circuit
pruned_ini_pauli

array([[2, 3, 3, 1],
       [2, 1, 0, 2],
       [3, 3, 0, 3],
       [0, 0, 1, 2],
       [2, 0, 3, 2],
       [2, 2, 0, 1],
       [0, 0, 2, 3],
       [3, 0, 2, 0],
       [0, 2, 1, 0],
       [0, 3, 0, 1],
       [0, 0, 1, 0],
       [0, 0, 2, 0],
       [0, 2, 3, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]], dtype=int64)

In [19]:
##calculate pruned circuit
pruned_circuit_state,_,pruned_grad_state_list,_=get_states_gradients(ini_pauli,ini_angles,n_parameters)
pruned_qfi_matrix=calc_qfi(pruned_circuit_state,pruned_grad_state_list)
#get eigenvalues and eigenvectors of QFI
pruned_eigvals,pruned_eigvecs=scipy.linalg.eigh(pruned_qfi_matrix)

In [16]:
pruned_n_parameters=len(construct_1d_parameters(ini_angles,pruned_ini_pauli)) #number of parameters of circuit


#non-zero eigenvalues of qfi
pruned_nonzero_eigvals=pruned_eigvals[pruned_eigvals>cutoff_eigvals]

#effective quantum dimension, i.e. number of independent directions when perturbing system
#is equivalent to parameter dimension (number of independent parameters of quantum state that can be represented by circuit) for random initial values of circuit
pruned_eff_quant_dim=len(pruned_nonzero_eigvals)

#zero eigenvalues of pruned circuit
pruned_n_zero_eigval=pruned_n_parameters-pruned_eff_quant_dim

In [17]:
print("Maximal effective dimension possible", 2**(n_qubits+1)-2)
print("Number parameters of circuit",pruned_n_parameters)
print("Effective quantum dimension G_C",pruned_eff_quant_dim)
print("Number of zero eigenvalues of QFI",pruned_n_zero_eigval)
#fraction of zero eigenvalues
pruned_redundancy=pruned_n_zero_eigval/pruned_n_parameters
print("redundancy",pruned_redundancy)

Maximal effective dimension possible 30
Number parameters of circuit 30
Effective quantum dimension G_C 30
Number of zero eigenvalues of QFI 0
redundancy 0.0
